In [43]:
import imp
import Data
import utils
import Predictors, Road_Graph, Neural_Network
imp.reload(Data)
imp.reload(Neural_Network)
imp.reload(utils)
imp.reload(Predictors)
imp.reload(Road_Graph)

from Road_Graph import *
from Predictors import *
from Data import *
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch
from torch import nn, autograd
import torch.nn.functional as F
import pickle

In [54]:
rnn_pd.predict([0,1,2,8,14,15,16,17])

(array([16, 11, 23]), Variable containing:
  0.0016
  0.5192
  0.4792
 [torch.FloatTensor of size 3])

In this part, we focus on a simple road map (rep as a graph below), and generate random path on it.
<img src="../img/naive_road.png" alt="Drawing" style="width: 400px;"/>

In [44]:
graph_dict = {}
graph_dict = graph_dict.fromkeys(range(24))
edges = [(i, i+1) for i in range(5)] \
        + [(i+6, i+7) for i in range(5)]\
                        +[(i+12, i+13) for i in range(5)]\
                     + [(i+18, i+19) for i in range(5)]\
                     + [(0,6),(6,12), (12,18)]\
                     + [(pair[0]+1, pair[1]+1) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+2, pair[1]+2) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+3, pair[1]+3) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+4, pair[1]+4) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+5, pair[1]+5) for pair in [(0,6),(6,12), (12,18)]]
for i,j in edges:
    if not graph_dict[i]:
        graph_dict[i] = [j]
    else:
        graph_dict[i].append(j)
    if not graph_dict[j]:
        graph_dict[j] = [i]
    else:
        graph_dict[j].append(i)

graph = Road_Graph(graph_dict)

# Markov Data

In [ ]:
def path_to_training_pair2(path):
    return path[1:], path[:-1]

def get_prob(x):
    nb = graph.neighbors(x)
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = model(W[x])
    #print('W_next:', W_next)
    #print(W_next.size(), Ws.size())
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    #print(W_next_rep.size(), Ws.size())
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    return prob

def get_loss(x, y):
    nb = graph.neighbors(x)
    #import pdb; pdb.set_trace()
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = model(W[x])
    #print('W_next:', W_next)
    #print(W_next.size(), Ws.size())
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    #print(W_next_rep.size(), Ws.size())
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    loss = -torch.log(prob[np.where(nb == y)])
    return loss

In [ ]:
dtype = torch.FloatTensor

data = random_walk_data(graph,10000)

im_D = 5
N = 24
h_D = 100


loss_list = []
W = Variable(torch.randn(N,im_D).type(dtype), requires_grad=True)
model = torch.nn.Sequential(
        torch.nn.Linear(im_D, h_D),
        torch.nn.ReLU(),
        torch.nn.Linear(h_D, im_D),
    )
for k in range(len(data)):
    path = data[k]
    x,y = path_to_training_pair2(path)
    
    optimizer = torch.optim.Adam(list(model.parameters()) + [W], lr=0.001) # remove [W] for a fixed random embeding
    for i in range(len(x)):
        loss = get_loss(x[i], y[i])
        loss_list.append(loss)
        
    if k%50 == 0:
        batch_loss = torch.sum(torch.cat(loss_list)) / len(loss_list)
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        loss_list = []
        print(batch_loss.data.numpy())
 

In [ ]:
def opt_log_loss():
    s = 0
    for i in range(24):
        n = len(graph.neighbors(i)) - 1
        s += -np.sum(np.ones(n) /n * np.log(np.ones(n)/n))
    return s/24
opt_log_loss()

In [ ]:
def opt_log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(np.ones(n)/n))
    return s/24
opt_log_loss()

In [ ]:
def log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(p_))
    return s/24
log_loss()

In [ ]:
def log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(p_))
    return s/24
log_loss()

In [ ]:
get_prob(1)

# Hist Dependent Data

In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.gru = nn.GRU(input_size, hidden_size, n_layers)
        self.lin = nn.Linear(hidden_size, output_size)
    
    def forward(self, inp, hidden):
        output, hidden = self.gru(inp.view(1, 1, self.input_size), hidden)
        output = self.lin(output) 
        return output, hidden        

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [ ]:
def path_loss(path):
    h = model.init_hidden()
    output_list = []
    for i in path:
        o, h = model(W[i], h)
        output_list.append(o)
    loss_list = []
    for i in range(len(path) - 1):
        x = path[i]
        y = path[i+1]
        nb = graph.neighbors(x)
        W_list = [W[j].view(-1,1) for j in nb]
        Ws = torch.cat(W_list, dim = 1)
        W_next = output_list[i]
        W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
        logit = torch.sum(W_next_rep*Ws, 0)
        prob = F.softmax(logit, dim = 0)
        loss_list.append(-torch.log(prob[np.where(nb == y)]))
    return torch.sum(torch.cat(loss_list))

def get_prob(path):
    h = model.init_hidden()
    for i in path:
        o, h = model(W[i], h)
    x = path[-1]
    nb = graph.neighbors(x)
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = o
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    return prob

In [ ]:
data = random_walk_data(graph, 10000, go_back=False)

In [ ]:
im_D = 50
N = 24
h_D = 100
dtype = torch.FloatTensor


W = Variable(torch.randn(N,im_D).type(dtype), requires_grad=True)
model = RNN(im_D, h_D, im_D)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # remove [W] for a fixed random embeding

loss_list = []
s = 0
for k in range(len(data)):
    path = data[k]
    loss = path_loss(path)
    loss_list.append(loss)    
    s += len(path)
    if k%50 == 0:
        batch_loss = torch.sum(torch.cat(loss_list)) / s
        s = 0
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        loss_list = []
        print(batch_loss.data.numpy())
 

In [ ]:
get_prob([1,2,3,4])

# Modulized Code

In [45]:
data = random_walk_data(graph, 200000, go_back=False)

In [48]:
rnn_pd = RNN_Predictor(graph)

In [49]:
rnn_pd.train(data,steps = 200000)

Loss after 1000 steps: 1.0716890686800273
Loss after 2000 steps: 0.9357152714814334
Loss after 3000 steps: 0.8858575648135681
Loss after 4000 steps: 0.8581178708161664
Loss after 5000 steps: 0.8427021602890676
Loss after 6000 steps: 0.8225637010670827
Loss after 7000 steps: 0.8267547350602289
Loss after 8000 steps: 0.8125967688591874
Loss after 9000 steps: 0.8079272696829592
Loss after 10000 steps: 0.8052598047192026
Loss after 11000 steps: 0.7995027268831563
Loss after 12000 steps: 0.7975608054380755
Loss after 13000 steps: 0.7942090446908708
Loss after 14000 steps: 0.796186477256597
Loss after 15000 steps: 0.7865377018104216
Loss after 16000 steps: 0.7854723164005479
Loss after 17000 steps: 0.7901354226348591
Loss after 18000 steps: 0.7879877107814146
Loss after 19000 steps: 0.777418080370983
Loss after 20000 steps: 0.7839760389860486
Loss after 21000 steps: 0.7833162295678966
Loss after 22000 steps: 0.7731230085421898
Loss after 23000 steps: 0.7744686836909603
Loss after 24000 steps

Loss after 191000 steps: 0.7606975305012229
Loss after 192000 steps: 0.7591090473306544
Loss after 193000 steps: 0.7575204856728606
Loss after 194000 steps: 0.7578850564756866
Loss after 195000 steps: 0.7610649959663642
Loss after 196000 steps: 0.7546343855786607
Loss after 197000 steps: 0.7627569611718199
Loss after 198000 steps: 0.7577663744496338
Loss after 199000 steps: 0.7594562984906723
Loss after 200000 steps: 0.7572365541767043


In [51]:
rnn_pd.predict([])

(array([1, 6]), Variable containing:
  0.4912
  0.5088
 [torch.FloatTensor of size 2])

Optimal total cross_entropy loss is ?

In [26]:
l = [0,1,2]

In [35]:
np.random.shuffle(l)

In [36]:
l

[2, 0, 1]

In [ ]:
None[0]

In [ ]:
data = np.random.shuffle(data)

In [ ]:
torch.sum(to_Variable([0]))